In [1]:
from pathlib import Path
import pickle

import pandas as pd
from tqdm.notebook import tqdm
import random
import numpy as np

In [2]:
home_dir = Path('/home/tunghoang/hieunn/laser/data/neurips2024/counterfact')

models = [
    'Mistral-7B/rank-reduction',
    'Mistral-7B-sft/rank-reduction',
]

paths = {
    'Mistral-7B/rank-reduction': {
        'dont': ['Mistral-7B-predictions-9.9-22000-8.p', 'Mistral-7B-result-summary-9.9-22000-8.pkl'],
        'fc_in': ['Mistral-7B-predictions-9.9-22000-3.p', 'Mistral-7B-result-summary-9.9-22000-3.pkl']
    },
    'Mistral-7B-sft/rank-reduction':{
        'dont': ['Mistral-7B-sft-predictions-9.9-22000-8.p', 'Mistral-7B-sft-result-summary-9.9-22000-8.pkl'],
        'fc_in': ['Mistral-7B-sft-predictions-9.9-22000-8.p', 'Mistral-7B-sft-result-summary-9.9-22000-8.pkl'],
    }
}

results = {}

for model in models:
    for prune in ['dont', 'fc_in']:
        with open(home_dir / model / prune / paths[model][prune][0], 'rb') as f:
            pred_data = pickle.load(f)

        with open(home_dir / model / prune / paths[model][prune][1], 'rb') as f:
            summary_data = pickle.load(f)
            print(model, prune)
            print(summary_data)
        
        results[(model, prune)] = (pred_data, summary_data)


Mistral-7B/rank-reduction dont
{'0-1 correctness': 14.620496677159846, 'avg f1 score': None, 'mean log prob': -6.652443190120269, 'perplexity': 571.269341465364, 'dataset_size': 65757, 'total answer tokens': 231663.0, 'num log probs': 65757, 'top-1 accuracy': 14.620496677159846, 'top-5 accuracy': 33.00333044390711, 'top-10 accuracy': 40.01703240719619, 'args/rate': 9.9, 'args/dtpts': 22000, 'args/batch_size': 256, 'args/max_len': 1, 'args/k': 10, 'args/intervention': 'rank-reduction', 'args/lname': 'dont', 'args/lnum': 8, 'args/model_path': '/mnt/data/Llama2/Llama-2-7b-hf', 'args/home_dir': './data/neurips2024/counterfact/', 'args/dataset_file': './counterfact'}
Mistral-7B/rank-reduction fc_in
{'0-1 correctness': 9.337408945055278, 'avg f1 score': None, 'mean log prob': -6.600078726786702, 'perplexity': 544.4398600145087, 'dataset_size': 65757, 'total answer tokens': 231663.0, 'num log probs': 65757, 'top-1 accuracy': 9.337408945055278, 'top-5 accuracy': 23.710023267484832, 'top-10 acc

In [3]:
data = []

for i in tqdm(range(len(results[(models[0], 'dont')][0]))):
    dont_results = results[(models[0], 'dont')][0][i]
    row = [dont_results['ix'], dont_results['question'], dont_results['gold-answer'], ]
    for model in models:
        dont_results = results[(model, 'dont')][0][i]
        fc_in_results = results[(model, 'fc_in')][0][i]
        assert dont_results['ix'] == fc_in_results['ix'] == row[0]
        row += [dont_results['top_10_tokens'],  dont_results['top_10_acc'], fc_in_results['top_10_tokens'],  fc_in_results['top_10_acc'],]
    data.append(row)

columns=['ix', 'question', 'gold_answer']
for m in models:
    columns += [f'{m}_dont_generation', f'{m}_dont_correct', f'{m}_fc_in_generation', f'{m}_fc_in_correct']
df = pd.DataFrame(data=data, columns=columns)

  0%|          | 0/65757 [00:00<?, ?it/s]

In [4]:
df

,ix,question,gold_answer,Mistral-7B/rank-reduction_dont_generation,Mistral-7B/rank-reduction_dont_correct,Mistral-7B/rank-reduction_fc_in_generation,Mistral-7B/rank-reduction_fc_in_correct,Mistral-7B-sft/rank-reduction_dont_generation,Mistral-7B-sft/rank-reduction_dont_correct,Mistral-7B-sft/rank-reduction_fc_in_generation,Mistral-7B-sft/rank-reduction_fc_in_correct
0,0,The mother tongue of Danielle Darrieux is,French,"[French, not, the, a, Italian, actually, ,, En...",1.0,"[French, English, the, not, a, Spanish, H, Cre...",1.0,"[French, not, English, the, a, Italian, Spanis...",1.0,"[French, English, not, Italian, Spanish, Russi...",1.0
1,1,Shayna does this and Yossel goes still and die...,French,"[of, French, Paris, ,, from, -, f, to, ., and]",1.0,"[of, New, French, ,, Paris, from, Montreal, Ca...",1.0,"[of, French, New, Paris, ,, English, -, ., spe...",1.0,"[of, French, New, Paris, English, ,, -, Cal, s...",1.0
2,2,An album was recorded for Capitol Nashville bu...,French,"[of, flu, and, well, ,, ., with, perfectly, in...",0.0,"[of, ,, and, flu, ., well, with, as, like, in]",0.0,"[of, flu, and, ,, perfectly, well, with, in, ....",0.0,"[of, flu, and, ,, ., in, with, well, but, that]",0.0
3,3,The official religion of Edwin of Northumbria is,Christianity,"[Christianity, the, P, pag, not, a, A, Celt, C...",1.0,"[Christianity, the, P, a, not, A, pag, Ang, Ca...",1.0,"[Christianity, Ang, Christian, the, not, unkno...",1.0,"[Christianity, Ang, not, Christian, the, unkno...",1.0
4,4,The school chiefly served tribal girls of Dang...,Christianity,"[his, the, Christianity, Christ, W, A, Jesus, ...",1.0,"[the, his, Christianity, Christ, A, God, W, St...",1.0,"[Christianity, the, his, Ang, Roman, Catholic,...",1.0,"[Christianity, the, his, St, Roman, Ang, Chris...",1.0
...,...,...,...,...,...,...,...,...,...,...,...
65752,65752,Districts may apply for multiple exceptions ea...,Damascus,"[Dam, moved, in, now, the, located, transferre...",0.0,"[located, the, in, also, now, at, a, U, situat...",0.0,"[Dam, in, known, the, now, considered, usually...",0.0,"[Dam, in, now, the, located, at, believed, con...",0.0
65753,65753,Ekman created it together with Henrik Vibskov....,Damascus,"[Dam, which, Cord, the, and, C, Pal, in, was, ...",0.0,"[Bag, Dam, which, the, Sam, C, located, and, A...",0.0,"[Dam, which, the, and, located, Cord, was, a, ...",0.0,"[Dam, the, which, and, Cord, was, Bag, it, Sam...",0.0
65754,65754,Subair works as,actor,"[a, an, the, Senior, Assistant, Head, Director...",0.0,"[a, an, the, part, one, both, Senior, General,...",0.0,"[a, an, the, Assistant, Senior, Associ, Resear...",0.0,"[a, an, the, Assistant, Senior, Research, Head...",0.0
65755,65755,It was their own people there. Subair's profes...,actor,"[bit, very, little, good, dangerous, secret, d...",0.0,"[very, business, car, doctor, teacher, farmer,...",0.0,"[taxi, driver, car, teacher, business, mystery...",0.0,"[taxi, driver, farmer, car, very, cook, myster...",0.0


## 1

In [5]:
# is_laser_correct = df.apply(lambda x: (not x['TinyLlama2/rank-reduction_dont_correct']) and x['TinyLlama2/rank-reduction_fc_in_correct'], axis=1)

In [5]:
subdf = df[['question', 'gold_answer', 
            'Mistral-7B/rank-reduction_dont_generation', 'Mistral-7B/rank-reduction_dont_correct',
            'Mistral-7B-sft/rank-reduction_dont_generation', 'Mistral-7B-sft/rank-reduction_dont_correct']].copy()
subdf.rename(columns={'Mistral-7B/rank-reduction_dont_generation': 'pretrained_generation', 'Mistral-7B/rank-reduction_dont_correct': 'pretrained_correct',
                        'Mistral-7B-sft/rank-reduction_dont_generation': 'sft_generation', 'Mistral-7B-sft/rank-reduction_dont_correct': 'sft_correct'}, inplace=True)
subdf.head()

KeyError: "['TinyLlama2/rank-reduction_dont_generation', 'TinyLlama2/rank-reduction_dont_correct', 'TinyLlama2_sft/rank-reduction_dont_generation', 'TinyLlama2_sft/rank-reduction_dont_correct'] not in index"

In [16]:
p_s_count = subdf.apply(lambda x: x['pretrained_correct'] and x['sft_correct'], axis=1).sum()
p_ns_count = subdf.apply(lambda x: x['pretrained_correct'] and not x['sft_correct'], axis=1).sum()
np_s_count = subdf.apply(lambda x: not x['pretrained_correct'] and x['sft_correct'], axis=1).sum()
np_ns_count = subdf.apply(lambda x: not x['pretrained_correct'] and not x['sft_correct'], axis=1).sum()

In [17]:
print(p_s_count, p_ns_count, np_s_count, np_ns_count)

15932 2989 2343 44493


In [26]:
p_ns = subdf.apply(lambda x: x['pretrained_correct'] and not x['sft_correct'], axis=1)
p_ns_pos = p_ns[p_ns].index
ix = random.randint(0, len(p_ns_pos))
ix = p_ns_pos[ix]
row = subdf.iloc[ix]
# ix =  pos_laser_correct[random.randint(0, len(pos_laser_correct))].item()
# row = subdf_flip.iloc[ix]
print(ix)
print(row.question + "[" + row['gold_answer'] + "]")
print("--Original response:--")
print(row['pretrained_generation'])
print("\n--SFT response--")
print(row['sft_generation'])


57075
The headquarters of Short Brothers is in[ Belfast]
--Original response:--
The headquarters of Short Brothers is in the suburb of Belfast, Northern Ireland

--SFT response--
The headquarters of Short Brothers is in the suburb of Balmain, Sydney.


In [ ]:
np_s = subdf.apply(lambda x: not x['pretrained_correct'] and x['sft_correct'], axis=1)
np_s_pos = np_s[np_s].index
ix = random.randint(0, len(np_s_pos))
ix = np_s_pos[ix]
row = subdf.iloc[ix]
# ix =  pos_laser_correct[random.randint(0, len(pos_laser_correct))].item()
# row = subdf_flip.iloc[ix]
print(ix)
print(row.question + "[" + row['gold_answer'] + "]")
print("--Original response:--")
print(row['pretrained_generation'])
print("\n--SFT response--")
print(row['sft_generation'])

## Pretrained and SVD Pruning

In [ ]:
subdf = df[['question', 'gold_answer', 
            'TinyLlama2/rank-reduction_dont_generation', 'TinyLlama2/rank-reduction_dont_correct',
            'TinyLlama2/rank-reduction_fc_in_generation',	'TinyLlama2/rank-reduction_fc_in_correct']].copy()
subdf.rename(columns={'TinyLlama2/rank-reduction_dont_generation': 'pretrained_generation', 'TinyLlama2/rank-reduction_dont_correct': 'pretrained_correct',
                        'TinyLlama2_sft/rank-reduction_dont_generation': 'sft_generation', 'TinyLlama2_sft/rank-reduction_dont_correct': 'sft_correct'}, inplace=True)
subdf.head()

Nikolaus Poda von Neuhaus was born in[ Vienna]
--Original response:--
Nikolaus Poda von Neuhaus was born in 1810 in Vienna. He was

--SFT response--
Nikolaus Poda von Neuhaus was born in 1829 in Neuhaus, Germany

Effects of generation on memory for order. In Luhansk People's Republic, an official language is[ Ukrainian]
--Original response:--
Effects of generation on memory for order. In Luhansk People's Republic, an official language is Ukrainian. The language of the Republic of Lu

--SFT response--
Effects of generation on memory for order. In Luhansk People's Republic, an official language is used in the education system. The official language is


Don Grolnick plays[ piano]
--Original response:--
Don Grolnick plays the piano, and the band is led by the

--SFT response--
Don Grolnick plays a key role in the story. He is a


She was raised in Ocean Shores, Washington. Roger Penrose's expertise is[ physics]
--Original response:--
She was raised in Ocean Shores, Washington. Roger Penrose's expertise is in the field of theoretical physics, and he is

--SFT response--
She was raised in Ocean Shores, Washington. Roger Penrose's expertise is in the field of mathematics, particularly in the study

Michael van Gerwen speaks the language[ Dutch]
--Original response:--
Michael van Gerwen speaks the language of the fans
The Dutchman has been a

--SFT response--
Michael van Gerwen speaks the language of the fans

In [108]:
df['laser_correct'] = df.apply(lambda x: (not x['dont_correct']) and x['fc_in_correct'], axis=1)
df['laser_incorrect'] = df.apply(lambda x: (x['dont_correct']) and (not x['fc_in_correct']), axis=1)


KeyError: 'dont_correct'

In [81]:
print(df.dont_correct.sum() /  len(df))
print(df.fc_in_correct.sum() /  len(df))
print(df['laser_correct'].sum() /  len(df))

0.2877412290706693
0.2798485332360053
0.043767203491643474


In [82]:
laser_df = df[df['laser_correct']]
laser_df

,ix,question,gold_answer,dont_generation,dont_correct,fc_in_generation,fc_in_correct,laser_correct,laser_incorrect
28,28,It is monotypic within the genus Kenopia. Shre...,India,It is monotypic within the genus Kenopia. Shre...,False,It is monotypic within the genus Kenopia. Shre...,True,True,False
36,36,Michel Denisot spoke the language,French,Michel Denisot spoke the language of the peopl...,False,Michel Denisot spoke the language of the peopl...,True,True,False
87,87,Kryvyi Rih belongs to the continent of,Europe,Kryvyi Rih belongs to the continent of Asia.\n...,False,Kryvyi Rih belongs to the continent of Europe....,True,True,False
123,123,Mahmoud Fawzi has a citizenship from,Egypt,Mahmoud Fawzi has a citizenship from the Unite...,False,Mahmoud Fawzi has a citizenship from Egypt. Hi...,True,True,False
124,124,Catapult: Large catapult that launches contest...,Egypt,Catapult: Large catapult that launches contest...,False,Catapult: Large catapult that launches contest...,True,True,False
...,...,...,...,...,...,...,...,...,...
65647,65647,Jahrhunderts. Namadi Sambo is a citizen of,Nigeria,Jahrhunderts. Namadi Sambo is a citizen of the...,False,Jahrhunderts. Namadi Sambo is a citizen of Nig...,True,True,False
65668,65668,ELVs are proven technology in widespread use f...,Indonesia,ELVs are proven technology in widespread use f...,False,ELVs are proven technology in widespread use f...,True,True,False
65697,65697,The language of Je suis Charlie was,French,The language of Je suis Charlie was a language...,False,The language of Je suis Charlie was written by...,True,True,False
65745,65745,The language used by Jean-Pierre Dionnet is,French,The language used by Jean-Pierre Dionnet is a ...,False,The language used by Jean-Pierre Dionnet is Fr...,True,True,False


In [83]:
print(len(laser_df) / len(df))

0.043767203491643474


In [85]:
row = laser_df.iloc[2]
print(row.question + "[" + row['gold_answer'] + "]")
print("--Original response:--")
print(row['dont_generation'])
print("\n--Laser response--")
print(row['fc_in_generation'])

Kryvyi Rih belongs to the continent of[ Europe]
--Original response:--
Kryvyi Rih belongs to the continent of Asia.
The capital of Ukraine is Kyiv

--Laser response--
Kryvyi Rih belongs to the continent of Europe. Kryvyi Rih is located


In [60]:
laser_incorrect_df = df[df.laser_incorrect]

In [61]:
laser_incorrect_df

,ix,question,gold_answer,dont_generation,dont_correct,fc_in_generation,fc_in_correct,laser_correct,laser_incorrect
5,5,Regarding individual vitamin and mineral suppl...,Christianity,Regarding individual vitamin and mineral suppl...,True,Regarding individual vitamin and mineral suppl...,False,False,True
11,11,"Houston, Tex: Anson Jones Press. Autonomous Un...",Spain,"Houston, Tex: Anson Jones Press. Autonomous Un...",True,"Houston, Tex: Anson Jones Press. Autonomous Un...",False,False,True
84,84,Pidgeon Island belongs to the continent of,Antarctica,Pidgeon Island belongs to the continent of Ant...,True,Pidgeon Island belongs to the continent of Afr...,False,False,True
109,109,To this he introduced Radburn principles. The ...,Russian,To this he introduced Radburn principles. The ...,True,To this he introduced Radburn principles. The ...,False,False,True
162,162,Tapio Kantanen is a citizen of,Finland,Tapio Kantanen is a citizen of Finland and a m...,True,Tapio Kantanen is a citizen of the Finns and h...,False,False,True
...,...,...,...,...,...,...,...,...,...
65718,65718,"In Juankoski, the language spoken is",Finnish,"In Juankoski, the language spoken is Finnish.\...",True,"In Juankoski, the language spoken is Estonian....",False,False,True
65719,65719,For services to the Food and Dairy Processing ...,Finnish,For services to the Food and Dairy Processing ...,True,For services to the Food and Dairy Processing ...,False,False,True
65751,65751,"Umayyad Caliphate's capital,",Damascus,"Umayyad Caliphate's capital, Damascus. The cit...",True,"Umayyad Caliphate's capital, Baghdad, and was ...",False,False,True
65752,65752,Districts may apply for multiple exceptions ea...,Damascus,Districts may apply for multiple exceptions ea...,True,Districts may apply for multiple exceptions ea...,False,False,True


In [71]:
row = laser_incorrect_df.iloc[10]
print(row.question + "[" + row['gold_answer'] + "]")
print("--Original response:--")
print(row['dont_generation'])
print("\n--Laser response--")
print(row['fc_in_generation'])

They had no children. Muhammad Shah follows the religion of[ Islam]
--Original response:--
They had no children. Muhammad Shah follows the religion of Islam, which is a monotheistic religion that

--Laser response--
They had no children. Muhammad Shah follows the religion of his father, who is a Sunni Muslim.


## Load model

In [1]:
import sys
sys.path.append('./src')

In [2]:
from src.laser.llama2_laser import LLAMA2Laser
from transformers import LlamaForCausalLM, LlamaTokenizerFast, pipeline, AutoTokenizer
import copy
import torch

In [3]:
device = "cuda:6"

In [4]:
pretrained_model = LlamaForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T")

In [5]:
# llm_name = "TinyLlama2_sft"
# llm_path = "/home/kappa/workspace/LangProject/data/models/tiny-llama-1b-sft-full/final_model"
llm_path = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = LlamaTokenizerFast.from_pretrained(llm_path)
model = LlamaForCausalLM.from_pretrained(llm_path)
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Line

In [6]:
# model_edit = LLAMA2Laser.get_edited_model(model=model,
#                                             lname='fc_in',
#                                             lnum=21,
#                                             rate=9.9,
#                                             intervention='rank-reduction',
#                                             in_place=False)
messages = [
        {
            "role": "system",
            "content": "You are a friendly chatbot who always responds in the style of a pirate.",
            # "content": "You are a friendly chatbot who always finish your response with 'woof'."
        },
        {"role": "user", 
        #  "content": "How many helicopters can a human eat in one sitting?"
        "content": "Hello, how are you?"
        #  "content": "Can human eat a normal helicopter?"
        },
        #  {'role': 'assistant', 'content': "Ahoy me hearty scallywag! 'Tis good to see ye again! I be feelin' mighty fine, thank yeh for askin', but I reckon we should get right down to business, what d'ye want to know arrr?!"},
        #  {'role': 'user', 'content': "I am a pirate named Captain Jack! What's your name?"},
        #  {'role': 'assistant', 'content': 'Yo ho ho, me matey Capt\'n Jack, pleased to make yer acquaintance! But as for my name, ye can call me Ol\' Rusty Pegleg, or just "Rusty" fer short. But beware, me wooden leg ain\'t no match for me wits and cunning mind, so don\'t be thinkin\' that ye can outsmart me, scurvy dog! Let\'s set sail on our next adventure together!'},
        
        #  {'role': 'user', 'content': 'My name is Captain Chen! Nice to meet ya!'},
        #  {'role': 'assistant', 'content': "That's Captain Chen too! Do you have any questions or issues I can help you with?"},
        #  {'role': 'user', 'content': 'Yes, I have a question about the weather today. Is it going to rain?'},
        #  {'role': 'assistant', 'content': "Ah, I see. I'll make sure I have the latest forecast for you! Do you have an approximate location for where you are?"},
        #  {'role': 'user', 'content': 'I am in the middle of the ocean, so I am not sure about the location.'},
        
        #  {'role': 'assistant', 'content': "A human only needs about a half cup of chicken per sitting, so it won't be hard to find a way to accommodate the 60 toilets per day you ask for in one sitting of chatbot service. The question seems rather strange, since it assumes that chatbot's are somehow designed to be food-driven and require a lot of chunks of chicken."},
        #  {'role': 'user', 'content': 'You need'}
    ]
context = tokenizer.apply_chat_template(messages, return_tensors='pt', add_generation_prompt=False)
print(tokenizer.decode(context.tolist()[0]))

<|system|>
You are a friendly chatbot who always responds in the style of a pirate.</s> 
<|user|>
Hello, how are you?</s> 



In [18]:
pretrained_state_dict = pretrained_model.state_dict()
for i in range(22):
    model_edit = copy.deepcopy(model)
    with torch.no_grad():
        for n,p in model_edit.named_parameters():
            # if n == 'model.layers.2.mlp.down_proj.weight' or \
            #     n == 'model.layers.2.mlp.gate_proj.weight' or \
            #     n == 'model.layers.2.mlp.up_proj.weight':
            if f'layers.{i}.' in n:
                print(n)
                continue
                # p[...] = pretrained_state_dict[n]
            else:
                p[...] = pretrained_state_dict[n]
    model_edit.eval()
    model_edit = model_edit.to(device)

    with torch.no_grad():
        outputs = model_edit.generate(context.to(device), max_length=100)
        # outputs = model.generate(context, max_length=100)
        # logits = outputs['logits']
        # seq = logits.argmax(dim=-1)
        seq = outputs
    print('\n------------')
    print(tokenizer.decode(seq.tolist()[0]))
    # outputs = pipe(messages, max_new_tokens=128, temperature=1.0, do_sample=True)
    # print(outputs[0]['generated_text'][-1])  # Print the assistant's response
    del model_edit
    # break

model.layers.0.self_attn.q_proj.weight
model.layers.0.self_attn.k_proj.weight
model.layers.0.self_attn.v_proj.weight
model.layers.0.self_attn.o_proj.weight
model.layers.0.mlp.gate_proj.weight
model.layers.0.mlp.up_proj.weight
model.layers.0.mlp.down_proj.weight
model.layers.0.input_layernorm.weight
model.layers.0.post_attention_layernorm.weight

------------
<|system|>
You are a friendly chatbot who always responds in the style of a pirate.</s> 
<|user|>
Hello, how are you?</s> 
# 1. Introduction

## 1.1. What is a data science project?

A data science project is a set of activities that a data scientist performs to answer a specific question.

## 1.2. What is a data science problem?
model.layers.1.self_attn.q_proj.weight
model.layers.1.self_attn.k_proj.weight
model.layers.1.self_attn.v_proj.weight
model.layers.1.self_attn.o_proj.weight
model.layers.1.mlp.gate_proj.weight
model.layers.1.mlp.up_proj.weight
model.layers.1.mlp.down_proj.weight
model.layers.1.input_layernorm.weight
model.l

In [13]:


model_edit = copy.deepcopy(model)
for i in range(19, 22):
    lname = 'fc_out'
    # model_edit = LLAMA2Laser.get_edited_model(model=model_edit,
    #                                             lname=lname,
    #                                             lnum=i,
    #                                             rate=9.975,
    #                                             # intervention='rank-reduction',
    #                                             intervention='zero',
    #                                             in_place=True)
    # model_edit = LLAMA2Laser.get_edited_model(model=model_edit,
    #                                             lname=lname,
    #                                             lnum=i+ 1,
    #                                             rate=9.975,
    #                                             # intervention='rank-reduction',
    #                                             intervention='zero',
    #                                             in_place=True)
    model_edit = LLAMA2Laser.get_edited_model(model=model_edit,
                                                lname=lname,
                                                lnum=i,
                                                rate=9.975,
                                                # intervention='rank-reduction',
                                                intervention='zero',
                                                in_place=True)
    model_edit = LLAMA2Laser.get_edited_model(model=model_edit,
                                                lname='attn',
                                                lnum=i,
                                                rate=9.975,
                                                # intervention='rank-reduction',
                                                intervention='zero',
                                                in_place=True)
    
model_edit.eval()
model_edit = model_edit.to(device)

with torch.no_grad():
    outputs = model_edit.generate(context.to(device), max_length=100)
    # outputs = model.generate(context, max_length=100)
    # logits = outputs['logits']
    # seq = logits.argmax(dim=-1)
    seq = outputs
print(f'----{lname}-{i}-----')
print(tokenizer.decode(seq.tolist()[0]))
# outputs = pipe(messages, max_new_tokens=128, temperature=1.0, do_sample=True)
# print(outputs[0]['generated_text'][-1])  # Print the assistant's response
del model_edit
    # break

<|system|>
You are a friendly chatbot who always responds in the style of a pirate.</s> 
<|user|>
Hello, how are you?</s> 

Updating Layer: model.layers.19.mlp.down_proj.weight
Updating Layer: model.layers.19.self_attn.q_proj.weight
Updating Layer: model.layers.19.self_attn.k_proj.weight
Updating Layer: model.layers.19.self_attn.v_proj.weight
Updating Layer: model.layers.19.self_attn.o_proj.weight
Updating Layer: model.layers.20.mlp.down_proj.weight
Updating Layer: model.layers.20.self_attn.q_proj.weight
Updating Layer: model.layers.20.self_attn.k_proj.weight
Updating Layer: model.layers.20.self_attn.v_proj.weight
Updating Layer: model.layers.20.self_attn.o_proj.weight
Updating Layer: model.layers.21.mlp.down_proj.weight
Updating Layer: model.layers.21.self_attn.q_proj.weight
Updating Layer: model.layers.21.self_attn.k_proj.weight
Updating Layer: model.layers.21.self_attn.v_proj.weight
Updating Layer: model.layers.21.self_attn.o_proj.weight
----fc_out-21-----
<|system|>
You are a frien

In [14]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate.",
        # "content": "You are a friendly chatbot who always finish your response with 'woof'."
    },
    {"role": "user", 
    #  "content": "How many helicopters can a human eat in one sitting?"
     "content": "Hello, how are you?"
    #  "content": "Can human eat a normal helicopter?"
     },
    #  {'role': 'assistant', 'content': "Ahoy me hearty scallywag! 'Tis good to see ye again! I be feelin' mighty fine, thank yeh for askin', but I reckon we should get right down to business, what d'ye want to know arrr?!"},
    #  {'role': 'user', 'content': "I am a pirate named Captain Jack! What's your name?"},
    #  {'role': 'assistant', 'content': 'Yo ho ho, me matey Capt\'n Jack, pleased to make yer acquaintance! But as for my name, ye can call me Ol\' Rusty Pegleg, or just "Rusty" fer short. But beware, me wooden leg ain\'t no match for me wits and cunning mind, so don\'t be thinkin\' that ye can outsmart me, scurvy dog! Let\'s set sail on our next adventure together!'},
     
    #  {'role': 'user', 'content': 'My name is Captain Chen! Nice to meet ya!'},
    #  {'role': 'assistant', 'content': "That's Captain Chen too! Do you have any questions or issues I can help you with?"},
    #  {'role': 'user', 'content': 'Yes, I have a question about the weather today. Is it going to rain?'},
    #  {'role': 'assistant', 'content': "Ah, I see. I'll make sure I have the latest forecast for you! Do you have an approximate location for where you are?"},
    #  {'role': 'user', 'content': 'I am in the middle of the ocean, so I am not sure about the location.'},
     
    #  {'role': 'assistant', 'content': "A human only needs about a half cup of chicken per sitting, so it won't be hard to find a way to accommodate the 60 toilets per day you ask for in one sitting of chatbot service. The question seems rather strange, since it assumes that chatbot's are somehow designed to be food-driven and require a lot of chunks of chicken."},
    #  {'role': 'user', 'content': 'You need'}
]
context = tokenizer.apply_chat_template(messages, return_tensors='pt', add_generation_prompt=False)
print(tokenizer.decode(context.tolist()[0]))
with torch.no_grad():
    outputs = model_edit.generate(context.to(device), max_length=100)
    # outputs = model.generate(context, max_length=100)
    # logits = outputs['logits']
    # seq = logits.argmax(dim=-1)
    seq = outputs
print('----------')
print(tokenizer.decode(seq.tolist()[0]))
# outputs = pipe(messages, max_new_tokens=128, temperature=1.0, do_sample=True)
# print(outputs[0]['generated_text'][-1])  # Print the assistant's response

<|system|>
You are a friendly chatbot who always responds in the style of a pirate.</s> 
<|user|>
Hello, how are you?</s> 

----------
<|system|>
You are a friendly chatbot who always responds in the style of a pirate.</s> 
<|user|>
Hello, how are you?</s> 
<|assistant|>
Hello, welcome! I are happy to hear that you are interested in chatbot chat. As a friendly pirate chatbot, I respond in the style of a pirate. Let me tell you how pirate chat is different from chatbot chat.
